---
title: NB Debugging ipywidgets in a Jupyter Notebook
description: >-
  A quick tip for debugging `ipywidgets` callbacks whren developing in Juptyer
  Notebook
date: '2022-12-02T11:05:08.443Z'
name: debugging-ipywidgets-jupyter-notebooks
oxa: oxa:DOHMeg040aVXqR51yjBy/Bj0JTf7ql7mFX2rqfz5M
subtitle: Stop your error messages from being eaten
short_title: ''
tags: []
thumbnail: thumbnails/debugging-ipywidgets-jupyter-notebooks.png
---

The `ipywidgets` library offers a big colleciton of standard widgets and suppports some really powerful third party widgets to like `ipympl`, `ipyleaflet` and more... So you can build some pretty sophisticated UI right in your Jupyter Notebook, then launch your app as a dashboard with `voila` or (ok, very soon!) create an interactive document with [Curvenote](https://curvenote.com).

When you UI starts getting bigger, developing and debugging can be tricky though for numerous reasons -- but the biggest thing is often access to `stdout` and `stderr` from the various callback functions that you end up creating. Luckily there i s a simple trick that helps solve this.

## A broken example

The following code has a problem, it will happily render my `ipympl` plot 👏 but the pick event does not work. When I pick on the plot nothing happens at all, no feedback, no error message.

In [0]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

plt.ioff()
fig, ax = plt.subplots(1, 1, figsize=(4,4))
plt.ion()

ax.plot(np.sin(np.random.random(100)),picker=True, pickradius=2)

selection = { "line": None }

def onpick(e):
    m = e.mouseevent
    e.artist.set__color('#DC143C')    
    e.artist.set_zorder(10)
    e.artist.set_linestyle('-')
    e.artist.set_marker('o')
    e.artist.set_markersize(e.artist.get_markersize() + 1)
    selection["line"] = e.artist
    

cid = fig.canvas.mpl_connect('pick_event', onpick)

display(fig.canvas)

```{figure} oxa:DOHMeg040aVXqR51yjBy/AzwMYrH2WdsNU5AqijkT.1
:name: gilDb8Ttsw
:align: center
:width: 50%
```

The problem is actually on the line
```
    e.artist.set__color('#DC143C')
```

Where there should be only one underscore, but the `stderr` generated in the callback function gets eaten. To solve this we need setup a way for `stderr` to get out to the notebook inteface. 
We can to this with `ipywidgets` and the `Output` widget!

## Debugging with an output widget

Here we need to do 3 things:

- We create an instance of an `Output` widget at the top of our code
- We `display(...)` our debug `Output` widget somewhere (doesn't have to the the same cell)
- We add a conditional `with context` block at the top of our event handlers

After that we get error messages showing up on our output display, and can also `print(...)` temporary debug messages there too while developing. To remove the output, we can just emove the code that creates the instance and the conditional context blocks can stay in our callbacks.

In [0]:
%matplotlib widget

# create an output widget!
w_debug_output = Output()

plt.ioff()

fig, ax = plt.subplots(1, 1, figsize=(4,4))
plt.ion()
ax.plot(np.sin(np.random.random(100)),picker=True, pickradius=2)

def onpick(e):
    with w_debug_output if w_debug_output else nullcontext():
        m = e.mouseevent
        e.artist.set__color('#DC143C')    
        e.artist.set_zorder(10)
        e.artist.set_linestyle('-')
        e.artist.set_marker('o')
        e.artist.set_markersize(e.artist.get_markersize() + 1)
        selection["line"] = e.artist

cid = fig.canvas.mpl_connect('pick_event', onpick)

display(fig.canvas)
display(w_debug_output)

```{figure} oxa:DOHMeg040aVXqR51yjBy/pnJpe04yDMVCRQ29Y6su.1
:name: ArelxFUKx3
:align: center
:width: 50%
```

This makes it pretty easy to fix our error, and gives you a chance to track errors while developing `ipywidgets` apps in your notebooks. Here's the result working below:

In [0]:
%matplotlib widget

# create an output widget!
w_debug_output = Output()

plt.ioff()

fig, ax = plt.subplots(1, 1, figsize=(4,4))
plt.ion()
ax.plot(np.sin(np.random.random(100)),picker=True, pickradius=2)

def onpick(e):
    with w_debug_output if w_debug_output else nullcontext():
        m = e.mouseevent
        e.artist.set_color('#DC143C')    
        e.artist.set_zorder(10)
        e.artist.set_linestyle('-')
        e.artist.set_marker('o')
        e.artist.set_markersize(e.artist.get_markersize() + 1)
        selection["line"] = e.artist

cid = fig.canvas.mpl_connect('pick_event', onpick)

display(fig.canvas)
display(w_debug_output)

```{figure} oxa:DOHMeg040aVXqR51yjBy/sfTtnAUzUSytH3JCmrrD.1
:name: SuTef2sxDy
:align: center
:width: 50%
```